#Dataset Sourced From: Sentiment140

The data is a CSV with emoticons removed. Data file format has 6 fields: 0-the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
1-the ID of the tweet (2087)
2-the date of the tweet (Sat May 16 23:58:44 UTC 2009)
3-the query (lyx). If there is no query, then this value is NO_QUERY.
4-the user that tweeted (robotickilldozr)
5-the text of the tweet (Lyx is cool)
http://help.sentiment140.com/for-students/
<font color="pink">-outline sklearn library

In [ ]:
import sklearn

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

In [ ]:
import tensorflow as tf
#enable GPU in "Hardware Acceleration" in the "Notebook Settings" in the "Edit" menu!
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print(('Found GPU at: {}'.format(device_name)))

Found GPU at: /device:GPU:0


In [ ]:
#enable "Share with anyone who has the link" in Google Drive, and copy the link

# https://drive.google.com/file/d/14Er7LdA9jG-Lz5mCyf9Ew5g-NsDEBcjs/view?usp=sharing
# remove the URL part and only copy the code for the link:
# e.g.: "!gdown --id 14Er7LdA9jG-Lz5mCyf9Ew5g-NsDEBcjs"

!gdown --id 14Er7LdA9jG-Lz5mCyf9Ew5g-NsDEBcjs

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=14Er7LdA9jG-Lz5mCyf9Ew5g-NsDEBcjs
To: /content/sentiment140.zip
100% 84.9M/84.9M [00:00<00:00, 285MB/s]


In [ ]:
!unzip sentiment140.zip

Archive:  sentiment140.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
cols = ['sentiment', 'id', 'date', 'query_string', 'user', 'text']
BASE_DIR = ''
df_tweets = pd.read_csv(os.path.join(BASE_DIR,'training.1600000.processed.noemoticon.csv'), header=None, encoding="latin-1", names=cols)

In [ ]:
df_tweets.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df_tweets.columns

Index(['sentiment', 'id', 'date', 'query_string', 'user', 'text'], dtype='object')

In [ ]:
df_tweets.tail()

,sentiment,id,date,query_string,user,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


#Clean Tweets

In [ ]:
def clean(raw):
  result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
  result = re.sub('&gt;', "", result)
  result = re.sub('&#x27;', "'", result)
  result = re.sub('&quot;', '"', result)
  result = re.sub('&#x2F;', ' ', result)
  result = re.sub('<p>', ' ', result)
  result = re.sub('</i>', '', result)
  result = re.sub('&#62', '', result)
  result = re.sub('<i>', ' ', result)
  result = re.sub("\n", '', result)
  return result

In [ ]:
df_tweets['clean_tweet'] = df_tweets.text.apply(func = clean)

In [ ]:
df_tweets.tail()

,sentiment,id,date,query_string,user,text,clean_tweet
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...,happy #charitytuesday @theNSPCC @SparksCharity...


#Split Dataset
This is called a stratified train-test split. ... This will be used by the train_test_split() function to ensure that both the train and test sets have the proportion of examples in each class that is present in the provided "y" array

In [ ]:
df_train, df_test = train_test_split(df_tweets, test_size=0.3, stratify=df_tweets['sentiment'], random_state=21)
#get the shape of train and test split.
print(df_train.shape, df_test.shape)

(1120000, 7) (480000, 7)


#TF-IDF Vector
Create a TF-IDF vector of the tweet text as defined above

In [ ]:
tfidf_vectorizer = TfidfVectorizer(lowercase=True, max_features=1000, stop_words=ENGLISH_STOP_WORDS)
#fit the object with the training data tweets
tfidf_vectorizer.fit(df_train.clean_tweet)

#transform the train and test data
train_idf = tfidf_vectorizer.transform(df_train.clean_tweet)
test_idf = tfidf_vectorizer.transform(df_test.clean_tweet)

TfidfVectorizer(max_features=1000,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

In [ ]:
df_tweets.nunique()

#Evaluating the LR Model

In [ ]:
# # f1 score on train data
# #print(sklearn.metrics.classification_report(y_true=df_train.sentiment, y_pred= predict_train))
# #print(sklearn.metrics.classification_report(y_true=df_test.sentiment, y_pred= predict_test))

In [ ]:
# pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer(lowercase=True,
#                                                      max_features=1000,
#                                                      stop_words= ENGLISH_STOP_WORDS)),
#                            ('model', LogisticRegression(n_estimators = 100))])

# # fit the pipeline model with the training data
# pipeline.fit(df_train.clean_tweet, df_train.Sentiment)

#Random Forest Model
create the object of Random Forest Model

fit the model with the training data

predict the label on the training data

predict the model on the test data

f1 score on training data

In [ ]:
model_RF = RandomForestClassifier(n_estimators=20)
model_RF.fit(train_idf, df_train.sentiment)
predict_train = model_RF.predict(train_idf)
predict_test = model_RF.predict(test_idf)

In [ ]:
#Prediction for the model data
#Prediction Training data and Test data
print(sklearn.metrics.precision_score(y_true=df_train.sentiment, y_pred=predict_train, pos_label=4))
print(sklearn.metrics.precision_score(y_true=df_test.sentiment, y_pred=predict_test, pos_label=4))

0.9068884962266369
0.7255874365332389


#Pipeline
When we use the fit() function with a pipeline object, both steps are executed.

In [ ]:
pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer(lowercase=True,
                                                     max_features=1000,
                                                     stop_words=ENGLISH_STOP_WORDS)),
                           ('model', RandomForestClassifier(n_estimators = 20))])

# fit the pipeline model with the training data
pipeline.fit(df_train.clean_tweet, df_train.sentiment)  #df_train.sentiment is the y value

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterwards', 'again',
                                                       'against', 'all',
                                                       'almost', 'alone',
                                                       'along', 'already',
                                                       'also', 'although',
                                                       'always', 'am', 'among',
                                                       'amongst', 'amoungst',
                                                       'amount', 'an', 'and',
                                                       'another', 'any',
                                                       'anyhow', 'anyone',
           

#Dump Model using Joblib
joblib takes any Python object. The object to store to disk.

joblib.dump() and joblib.load() are based on the Python pickle serialization model, which can be used when loading a serialized object with joblib.load().

#Test Model on SampleTweet
#loading saved model from above